In [1]:
import pandas as pd
import numpy as np

## CSV Importation and transformation

We import the csv file

In [2]:
df = pd.read_csv('ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv')
df.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,...,"QUXVlc3Rpb246MTk0 - Si vous avez été amené à préparer votre retraite, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :","QUXVlc3Rpb246MTk1 - Si vous avez été amené à demander un remboursement de soins de santé, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :","QUXVlc3Rpb246MTk2 - Si vous avez été amené à faire une demande d'aide pour une situation de handicap, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :","QUXVlc3Rpb246MTk4 - Si vous avez été amené à créer une entreprise, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :","QUXVlc3Rpb246MTk5 - Si vous avez été amené à recruter du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :","QUXVlc3Rpb246MjAw - Si vous avez été amené à former du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :","QUXVlc3Rpb246MjAx - Si vous avez été amené à rémunérer du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :","QUXVlc3Rpb246MjAy - Si vous avez été amené à mettre fin à votre activité, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :","QUXVlc3Rpb246MjAz - Si vous avez été amené à recruter une personne portant un handicap, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :",QUXVlc3Rpb246MTg5 - Y a-t-il d'autres points sur l'organisation de l'Etat et des services publics sur lesquels vous souhaiteriez vous exprimer ?
0,UHJvcG9zYWw6NjllZDU1NzItMWUyMS0xMWU5LTk0ZDItZm...,4-3,Les avantages,2019-01-22 09:40:47,2019-01-22 09:40:47,NaN,False,NaN,VXNlcjo4Mjc4NzQxYS0xZTFkLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Les anciens Présidents, anciens membres du gou..."
1,UHJvcG9zYWw6NzlmMGY3YjItMWUyMS0xMWU5LTk0ZDItZm...,4-4,suppression des départements,2019-01-22 09:41:14,2019-01-22 09:41:14,NaN,False,NaN,VXNlcjo3ZTVjYTUwMi0xZDZlLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We correct the date to remove the HH:MM part

In [3]:
def date_curation (date):
    try: 
        return(date[:10])
    except TypeError:
        return(None)

df['createdAt']=df['createdAt'].apply(date_curation)
df['publishedAt']=df['createdAt'].apply(date_curation)
df['updatedAt']=df['updatedAt'].apply(date_curation)

We change the structure of the file to have one row per question answered

In [4]:
df_elastic = pd.DataFrame()
colnumber = 11
for question in (df.columns[colnumber:]):
    df_temp = df.iloc[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, colnumber]].copy().rename(columns={question : 'answer'})
    df_temp["question"]=question[20:]
    df_elastic = pd.concat([df_elastic, df_temp])
    print (question[20:])
    print(colnumber)
    colnumber +=1
df_elastic = df_elastic.reset_index()

Que pensez-vous de l'organisation de l'Etat et des administrations en France ? De quelle manière cette organisation devrait-elle évoluer ?
11
Selon vous, l'Etat doit-il aujourd'hui transférer de nouvelles missions aux collectivités territoriales ?
12
Si oui, lesquelles ?
13
Estimez-vous avoir accès aux services publics dont vous avez besoin ?
14
Si non, quels types de services publics vous manquent dans votre territoire et qu'il est nécessaire de renforcer ?
15
Quels nouveaux services ou quelles démarches souhaitez-vous voir développées sur Internet en priorité ?
16
Avez-vous déjà utilisé certaines de ces nouvelles formes de services publics ?
17
Si oui, en avez-vous été satisfait ?
18
Quelles améliorations préconiseriez-vous ?
19
Quand vous pensez à l'évolution des services publics au cours des dernières années, quels sont ceux qui ont évolué de manière positive ?
20
Quels sont les services publics qui doivent le plus évoluer selon vous ?
21
Connaissez-vous le "droit à l'erreur", c'es

## Sentiment analysis component using textblob

In [5]:
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

In [6]:
def sentiment_analysis(expression):
    blob = TextBlob(expression, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
    return (blob.sentiment[0], blob.sentiment[1])

In [7]:
df_elastic['answer'] = df_elastic['answer'].fillna("")
df_elastic['positivity'], df_elastic['intensity'] = zip(*df_elastic['answer'].map(sentiment_analysis))

In [8]:
df_elastic['sentiment'] = df_elastic['positivity']*df_elastic['intensity']

In [9]:
df_elastic.tail()

,index,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,authorZipCode,answer,question,positivity,intensity,sentiment
2469715,74835,UHJvcG9zYWw6ZTE1ZTg3YTEtNDE4OS0xMWU5LWJmNTYtZm...,4-85878,"SIMPLIFICATION, PROXIMITE et MODERNISATION",2019-03-08,2019-03-08,None,False,NaN,VXNlcjo5ZmRjYzk4OC00MTdkLTExZTktYmY1Ni1mYTE2M2...,Citoyen / Citoyenne,38110.0,,Y a-t-il d'autres points sur l'organisation de...,0.000,0.000,0.000000
2469716,74836,UHJvcG9zYWw6ZTlkYzVmMmYtNDE4OS0xMWU5LWJmNTYtZm...,4-85879,plus d'appel systématique des décisions de jus...,2019-03-08,2019-03-08,None,False,NaN,VXNlcjpjMzdhMjZjZC0yNGI5LTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,38920.0,il faut réformer la loi pour que l'appel en ju...,Y a-t-il d'autres points sur l'organisation de...,0.163,0.285,0.046455
2469717,74837,UHJvcG9zYWw6MGIyMjcyZTYtNDE4YS0xMWU5LWJmNTYtZm...,4-85880,Service public,2019-03-08,2019-03-08,None,False,NaN,VXNlcjoxZDA4NWNkZS00MTg2LTExZTktYmY1Ni1mYTE2M2...,Citoyen / Citoyenne,33200.0,,Y a-t-il d'autres points sur l'organisation de...,0.000,0.000,0.000000
2469718,74838,UHJvcG9zYWw6MzE4Nzk3ODQtNDE4YS0xMWU5LWJmNTYtZm...,4-85881,indépendance numérique de l'Etat en question,2019-03-08,2019-03-08,None,False,NaN,VXNlcjo4MDk4YWZhZi0xZjI3LTExZTktOTRkMi1mYTE2M2...,Élu / élue et Institution,81150.0,Sortir de la dépendance et du coût exorbitant ...,Y a-t-il d'autres points sur l'organisation de...,0.155,0.110,0.017050
2469719,74839,UHJvcG9zYWw6Mzg4MjBiYTctNDE4YS0xMWU5LWJmNTYtZm...,4-85882,Décentraliser ! Décentraliser !,2019-03-08,2019-03-08,None,False,NaN,VXNlcjo2MjUyZjUwZi00MTdmLTExZTktYmY1Ni1mYTE2M2...,Citoyen / Citoyenne,75017.0,,Y a-t-il d'autres points sur l'organisation de...,0.000,0.000,0.000000


In [10]:
df_elastic.to_csv("organisation_enriched.csv", encoding='utf-8', index=False)

## Sentiment analysis component using translation and flair

cf democratie pour plus d'info

## Geocoding and source adding

geocoding is a file that has been downloaded using http://www.nosdonnees.fr/wiki/index.php/Image:EUCircos_Regions_departements_circonscriptions_communes_gps.csv.gz

In [12]:
df_elastic_clean = pd.read_csv('organisation_enriched.csv', encoding='utf-8', engine='python')

In [13]:
df_elastic_clean.columns

Index(['index', 'id', 'reference', 'title', 'createdAt', 'publishedAt',
       'updatedAt', 'trashed', 'trashedStatus', 'authorId', 'authorType',
       'authorZipCode', 'answer', 'question', 'positivity', 'intensity',
       'sentiment'],
      dtype='object')

In [14]:
try:
    df_elastic = df_elastic_clean.drop(['departement','city','latitude','longitude'], axis=1)
except:
    pass
def code_correction(code):
    if len(code)==4:
        return(str(code[:-2]+'000'))
    if len(code)==6:
        return(str('0'+code[:-2]))
    return(str(code[:-2]))

df_elastic['authorZipCode']=df_elastic['authorZipCode'].astype(str)
df_elastic['authorZipCode']=df_elastic['authorZipCode'].apply(code_correction)

In [15]:
geocoding = pd.read_csv('geocoding_corrected.csv', sep=';')

geocoding['codes_postaux']=geocoding['codes_postaux'].astype(str)

def code_correction(code):
    if len(code)==2:
        return(code+'000')
    if len(code)==4:
        return('0'+code)
    return(code)

geocoding['codes_postaux']=geocoding['codes_postaux'].apply(code_correction)

df_elastic['authorZipCode']=df_elastic['authorZipCode'].astype(str)
geocoding= (geocoding[["nom_departement","nom_commune","codes_postaux","latitude","longitude"]]
            .sort_values(by=["codes_postaux"], axis=0)
            .drop_duplicates(["codes_postaux"], 'first')
            .rename(columns={"nom_departement":"departement", "nom_commune":"city"}))
geocoding=geocoding.fillna(method='pad')
df_elastic = pd.merge(df_elastic, geocoding, how='left', left_on = 'authorZipCode', right_on='codes_postaux')

In [17]:
df_elastic[df_elastic['latitude'].isna()]["authorZipCode"].value_counts()

0          1716
87100      1452
37100      1320
38920      1254
94210      1056
76100      1023
99000       990
72100       891
06160       858
76620       858
83100       792
13540       792
99999       792
57950       759
57970       660
13290       660
59240       660
97490       627
57155       627
80090       594
98800       594
93210       561
68200       528
37200       462
33970       429
50130       429
13080       396
57280       396
97432       396
66100       363
           ... 
921000       33
9355429      33
921120       33
664740       33
05862        33
91500        33
1040033      33
778125       33
15280        33
29492        33
762230       33
8            33
751116       33
13104        33
02333        33
34133        33
01206        33
99422        33
50931        33
971          33
03409        33
01083        33
302920       33
02343        33
29485        33
98735        33
610000       33
11456        33
2500334      33
35100        33
Name: authorZipCode, Len

In [18]:
df_elastic['source']='Grand débat - organisation de l\'Etat et des services publics'
df_elastic.to_csv("organisation_enriched.csv", encoding='utf-8', index=False)

## json writing for elasticsearch

In [10]:
df_elastic = pd.read_csv('democratie_enriched.csv', encoding='utf-8', engine='python')

In [19]:
f = open("organisation_base_pre_es.json", "w")

for row in df_elastic.iterrows():
    row[1].to_json(f, orient='index', force_ascii=True)
    f.write("\n")
f.write("\n")

1

## Ingestion in elasticsearch

In [20]:
try:
    df_elastic['trashed'] = np.where((df_elastic['trashed'] =='True') & (df_elastic['trashed'] =='False'),True,False)
except:
    pass

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [21]:
with open('./organisation_base_pre_es.json', 'r') as r, open('./organisation_base_es.json', 'w') as w:    
    for num, line in enumerate(r):
        if line[0]!='[' and line[0]!=']':
            newline = '{"index":{"_id":'+ '"o_' + str(num)+'"}}'+'\n'
            if line[0]=='\n':
                newline='\n'
            elif line[-2]== ',':
                newline += line[:-2] + "\n" if "\n" in line else line[:-1]
            else:
                newline += line[:-1] + "\n" if "\n" in line else line[:-1]
        else:
            newline = ''               
        w.write(newline)